In [ ]:
import cv2
import numpy as np
import os

# Global variables to store the points
points = []

# Mouse callback function to select points
def select_points(event, x, y, flags, param):
    global points
    if event == cv2.EVENT_LBUTTONDOWN:  # Left-click to add points
        if len(points) < 6:  # Limit the number of points to 6
            points.append((x, y))
            cv2.circle(image_copy, (x, y), 5, (0, 0, 255), -1)  # Draw a small red circle on selected points
            cv2.imshow("Select Region", image_copy)
        else:
            print("Maximum of 6 points can be selected. Press 'q' to confirm.")

# Create directories for saving outputs
os.makedirs('Masks', exist_ok=True)
os.makedirs('Output', exist_ok=True)

# Load the image
image_path = 'Square Input/rotated_135/20.png'  # Update with your image path
image = cv2.imread(image_path)

# Check if the image was loaded properly
if image is None:
    print(f"Error: Unable to open image file {image_path}. Check the path and file format.")
else:
    # Resize the image to have width 512px while keeping aspect ratio
    target_width = 512
    height, width = image.shape[:2]
    aspect_ratio = height / width
    target_height = int(target_width * aspect_ratio)
    image = cv2.resize(image, (target_width, target_height))
    image_copy = image.copy()

    # Display the resized image in a window
    cv2.imshow("Select Region", image)
    cv2.setMouseCallback("Select Region", select_points)

    # Wait until 'q' is pressed to finish the selection
    print("Select points by left-clicking (maximum 6 points), and press 'q' to confirm.")
    while True:
        cv2.imshow("Select Region", image_copy)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Close the image window after selecting points
    cv2.destroyAllWindows()

    # Ensure at least 3 points are selected to form a polygon
    if len(points) < 3:
        print("At least 3 points are required to form a polygon.")
    else:
        print(f"Selected points: {points}")  # Debugging: print selected points

        # Create a mask for the polygonal region of interest (ROI)
        mask = np.zeros(image.shape[:2], dtype=np.uint8)  # Mask with the same size as the resized image
        points_np = np.array(points, np.int32)  # Convert points to numpy array
        points_np = points_np.reshape((-1, 1, 2))  # Reshape for fillPoly function

        # Fill the polygon on the mask
        cv2.fillPoly(mask, [points_np], 255)  # Fill the polygon with white (255)

        # Debugging: visualize the mask
        cv2.imshow("Mask", mask)
        cv2.waitKey(0)

        # Extract the selected region using the mask
        selected_region = cv2.bitwise_and(image, image, mask=mask)

        # Debugging: visualize the selected region
        cv2.imshow("Selected Region (Raw)", selected_region)
        cv2.waitKey(0)

        # Create a white background
        white_background = np.ones_like(image) * 255  # White background image

        # Place the selected region onto the white background
        background_with_selection = cv2.bitwise_or(white_background, selected_region)

        # Draw the polygonal boundary on the selected region with a thinner line
        cv2.polylines(background_with_selection, [points_np], isClosed=True, color=(0, 0, 0), thickness=5)

        # Show the final output
        cv2.imshow("Selected Region with Polygon", background_with_selection)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # Define output paths for mask and final output
        mask_output_path = 'Square Mask/150_mask.png'  # Path for saving the mask51
        final_output_path = 'Square Output/150.png'    # Path for saving the final output

        # Save the mask
        cv2.imwrite(mask_output_path, mask)
        print(f"Mask image saved at: {mask_output_path}")

        # Save the final output
        cv2.imwrite(final_output_path, background_with_selection)
        print(f"Final image saved at: {final_output_path}")


Select points by left-clicking (maximum 6 points), and press 'q' to confirm.
